In [1]:
import os
import pandas as pd
import glob

from pathlib import Path

In [2]:
dir_path = Path(os.getcwd())
data_dir = str(os.path.join(dir_path.parent,'data'))
balance_files = glob.glob(data_dir + "/poker_balances*.csv")
hands_files = glob.glob(data_dir + "/poker_hands*.csv")
table_files = glob.glob(data_dir + "/poker_table_info*.csv")

def get_hands_df():
    hands_dfs=[]

    for file in hands_files:
        new_df = pd.read_csv(file)
        hands_dfs.append(new_df)

    hands = pd.concat(hands_dfs)
    return hands
    
def get_games_df():
    balance_dfs=[]
    
    for file in balance_files:
        new_df = pd.read_csv(file)
        balance_dfs.append(new_df)
        
    balances = pd.concat(balance_dfs)
    return balances

def get_table_df():
    table_dfs=[]
    
    for file in table_files:
        new_df = pd.read_csv(file)
        table_dfs.append(new_df)
        
    tables = pd.concat(table_dfs)
    return tables


In [10]:
pd.set_option('display.max_columns', None)  
hands_base = get_hands_df()
games_base = get_games_df()
tables = get_table_df()

games = pd.merge(games_base,tables)
hands = pd.merge(hands_base,games)

In [11]:
tables

,table_id,player_types
0,1,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...


In [12]:
games

,table_id,game_id,player_name,player_type,game_result,game_reason,blind_type,beginning_balance,game_start_balance,game_end_balance,game_net_change,player_types
0,1,1,players_3,AlwaysCallPlayer,won,won_game,None,100000,100000,100000,0,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
1,1,2,players_3,AlwaysCallPlayer,won,won_game,None,100000,100000,100000,0,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
2,1,3,players_3,AlwaysCallPlayer,won,won_game,small,100000,100000,100000,0,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
3,1,4,players_3,AlwaysCallPlayer,won,won_game,big,100000,100000,100000,0,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
4,1,5,players_3,AlwaysCallPlayer,won,won_game,None,100000,100000,100000,0,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
...,...,...,...,...,...,...,...,...,...,...,...,...
595,1,96,players_2,AlwaysCallPlayer,won,won_game,None,100000,100024,100024,0,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
596,1,97,players_2,AlwaysCallPlayer,won,won_game,None,100000,100024,100024,0,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
597,1,98,players_2,AlwaysCallPlayer,won,won_game,None,100000,100024,100024,0,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
598,1,99,players_2,AlwaysCallPlayer,won,won_game,None,100000,100024,100024,0,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...


In [13]:
hands

,table_id,game_id,player_name,player_type,bet_number,opponents,call,current,final,pot,allowed,hand1,hand2,community1,community2,community3,community4,community5,game_result,game_reason,blind_type,beginning_balance,game_start_balance,game_end_balance,game_net_change,player_types
0,1,1,players_3,AlwaysCallPlayer,1,5,10,0,10.0,35,True,10-clubs,7-clubs,Z-N/A,Z-N/A,Z-N/A,Z-N/A,Z-N/A,won,won_game,None,100000,100000,100000,0,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
1,1,1,players_3,AlwaysCallPlayer,2,5,10,10,20.0,90,True,10-clubs,7-clubs,Z-N/A,Z-N/A,Z-N/A,Z-N/A,Z-N/A,won,won_game,None,100000,100000,100000,0,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
2,1,1,players_3,AlwaysCallPlayer,3,5,10,20,30.0,150,False,10-clubs,7-clubs,Z-N/A,Z-N/A,Z-N/A,Z-N/A,Z-N/A,won,won_game,None,100000,100000,100000,0,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
3,1,1,players_3,AlwaysCallPlayer,4,5,0,30,30.0,180,True,10-clubs,7-clubs,4-diamonds,7-hearts,8-clubs,Z-N/A,Z-N/A,won,won_game,None,100000,100000,100000,0,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
4,1,1,players_3,AlwaysCallPlayer,5,5,10,30,40.0,210,True,10-clubs,7-clubs,4-diamonds,7-hearts,8-clubs,Q-hearts,Z-N/A,won,won_game,None,100000,100000,100000,0,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3980,1,100,players_2,AlwaysCallPlayer,2,5,0,10,10.0,60,True,3-diamonds,5-hearts,4-spades,7-diamonds,J-spades,Z-N/A,Z-N/A,won,won_game,small,100000,100024,100024,0,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
3981,1,100,players_2,AlwaysCallPlayer,3,5,0,10,10.0,60,True,3-diamonds,5-hearts,4-spades,7-diamonds,J-spades,Q-spades,Z-N/A,won,won_game,small,100000,100024,100024,0,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
3982,1,100,players_2,AlwaysCallPlayer,4,5,0,10,10.0,60,True,3-diamonds,5-hearts,4-spades,7-diamonds,J-spades,K-diamonds,Q-spades,won,won_game,small,100000,100024,100024,0,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
3983,1,100,players_2,AlwaysCallPlayer,5,5,10,10,20.0,80,True,3-diamonds,5-hearts,4-spades,7-diamonds,J-spades,K-diamonds,Q-spades,won,won_game,small,100000,100024,100024,0,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...


# Basic Game Statistics

Number of Games played in the simulation

In [14]:
number_of_games = len(set(games.game_id))
number_of_games

100

Players Mean number of chips gained and variance

In [7]:
chips_gained_per_game = games.groupby('player_name').mean('game_net_change')[['game_net_change']].reset_index()
variance_calculation = games[['player_name','game_net_change']]
variance_calculation['game_net_change'] = variance_calculation['game_net_change'] ** 2
expectation_squared = variance_calculation.groupby('player_name').mean('game_net_change').reset_index()
merged_expectations = pd.merge(expectation_squared,chips_gained_per_game,on='player_name')
merged_expectations['variance'] = merged_expectations['game_net_change_x'] - merged_expectations['game_net_change_y']
summary_stats = merged_expectations[['player_name','game_net_change_y','variance']]
summary_stats.columns = ['player_name','mean','variance']
summary_stats['games'] = number_of_games
summary_stats

c:\users\silbe\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,player_name,mean,variance,games
0,players_1,0.24,1.36,100
1,players_2,0.24,1.36,100
2,players_3,0.24,1.36,100
3,players_4,0.24,1.36,100
4,players_5,0.24,1.36,100
5,players_6,-1.20,41.20,100


Top 10 Best Card Hands Dealt by Mean Number of Chips Gained 

In [40]:
two_card_hands = hands[['game_id','player_name','player_type','hand1','hand2','game_net_change']]
two_card_hands = two_card_hands.drop_duplicates()
best_two_card_hands = two_card_hands.groupby(['player_name','hand1','hand2']).mean().reset_index()[['player_name','hand1','hand2','game_net_change']]
best_two_card_hands = best_two_card_hands.sort_values(by=['player_name','game_net_change'], ascending=[True,False])
best_two_card_hands.groupby('player_name').head(10)

,player_name,hand1,hand2,game_net_change
45,players_1,4-hearts,J-spades,8.0
67,players_1,6-hearts,Q-clubs,8.0
16,players_1,2-diamonds,6-spades,4.0
39,players_1,4-diamonds,5-diamonds,4.0
0,players_1,10-clubs,10-diamonds,0.0
1,players_1,10-clubs,8-spades,0.0
2,players_1,10-clubs,A-diamonds,0.0
3,players_1,10-clubs,Q-clubs,0.0
4,players_1,10-diamonds,K-diamonds,0.0
5,players_1,10-diamonds,Q-diamonds,0.0
